In [ ]:
import math
import numpy as np
import pandas as pd

import cv2
import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.utils import to_categorical

In [ ]:
df = pd.read_csv("C:/Users/Pierre/Desktop/final_final_try/fer2013.csv")
print(df.shape)
df.head()

In [ ]:
df.emotion.unique()#bytb3 el 7agat el unique mara wa7da 

In [ ]:
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}#by7wl el klam le arkam

In [ ]:
df.emotion.value_counts()#kol class fi ad eh

In [ ]:
math.sqrt(len(df.pixels[0].split(' ')))# dah ely bytl3li el dmiensions. bymsk el pixels y7ot mabenhom space we ygib el len, el sqrt(len)

In [ ]:
fig = pyplot.figure(1, (14, 14))

k = 0
for label in sorted(df.emotion.unique()):
    for j in range(7):
        px = df[df.emotion==label].pixels.iloc[k]
        px = np.array(px.split(' ')).reshape(48, 48).astype('float32')

        k += 1
        ax = pyplot.subplot(7, 7, k)
        ax.imshow(px, cmap = 'gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(emotion_label_to_text[label])
        pyplot.tight_layout()

#bytb3 sample mn kol emotion


In [ ]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48).astype('float32'))
#by5li el pixel vector 2d matrix size 48,48
img_array = np.stack(img_array, axis = 0)
#by7ot el matrices fi stack

In [ ]:
img_array.shape

In [ ]:
img_features = []

for i in range(len(img_array)):
    temp = cv2.cvtColor(img_array[i], cv2.COLOR_GRAY2RGB)
    #by7wl el sora le 3 channels 3lshan el input dimensions tzbot
    img_features.append(temp)

img_features = np.array(img_features)
print(img_features.shape)

In [ ]:
pyplot.imshow(img_features[0].astype(np.uint8))

In [ ]:
le = LabelEncoder()

img_labels = le.fit_transform(df.emotion)
img_labels = to_categorical(img_labels)
img_labels.shape
################################################33
#hna by3ml one hot encoding
#1000000-->class 0-->angry
#0001000-->class 4-->happy

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
#hna by3ml mapping maben el classes wel hot enconfind
#0 class--->angry// text-->numbers angry--->0       angry(0)---> 0
#                                                   happy(1)--> 1
#zip dih bt pair kol label bel int value bt3to 
#dict-->data structure el dictionary
print(le_name_mapping)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(img_features, 
                                                      img_labels, 
                                                      shuffle = True, 
                                                      stratify = img_labels, 
                                                      test_size = 0.1, 
                                                      random_state = 42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

#b2sm el data

In [ ]:
del df
del img_features
del img_labels

#bms7 eli msh m7tago tani

In [ ]:
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

In [ ]:
X_train = X_train / 255.
X_valid = X_valid / 255.
#normalize pixels

In [ ]:
vgg = tf.keras.applications.VGG19(weights = 'imagenet',
                                  include_top = False,
                                  input_shape = (48, 48, 3))

#vgg16 model-->weights mn imagenet, include_top-->el dense layers bta3t el vgg msh han5ofha

In [ ]:
vgg.summary()

In [ ]:
def build_model(bottom_model, classes):
    model = bottom_model.layers[-2].output
    model = GlobalAveragePooling2D()(model)
    model = Dense(classes, activation = 'softmax', name = 'out_layer')(model)
    
    # el output layer hia el layer -2 mn el vgg 
    #mwslin el -2 dih bel globalavgpool-->avg filter zai el image bas hay3mlha 3la el image
    # kolha msh kernel we ytl3 value wa7da
    # el global mtwsl bel output layer
    return model

In [ ]:
head = build_model(vgg, num_classes)

model = Model(inputs = vgg.input, outputs = head)

print(model.summary())

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_accuracy', 
                               min_delta = 0.00005, # el val_acc lazm t3di el min 3lshan yb2a improvment
                               patience = 11,# 11 no improvment wara b3d--->stop
                               verbose = 1, #flag
                               restore_best_weights = True,)

lr_scheduler = ReduceLROnPlateau(monitor = 'val_accuracy', 
                                 factor = 0.5, 
                                 patience = 7,
                                 min_lr = 1e-7,
                                 verbose = 1,)

callbacks = [early_stopping,lr_scheduler,]

In [ ]:
#data augmentation 3lshan ys3ad yw2f el ovwerfitting
train_datagen = ImageDataGenerator(rotation_range = 15,
                                   width_shift_range = 0.15,
                                   height_shift_range = 0.15,
                                   shear_range = 0.15,
                                   zoom_range = 0.15,
                                   horizontal_flip = True,)
#dih filters btt3ml 3la el swr
train_datagen.fit(X_train)

In [ ]:
batch_size = 32 
epochs = 25
optims = optimizers.Adam(learning_rate = 0.0001)

model.compile(loss = 'categorical_crossentropy',
              optimizer = optims,
              metrics = ['accuracy'])

In [ ]:
with tf.device('/GPU:0'):
    history = model.fit(
        train_datagen.flow(X_train, y_train, batch_size=batch_size),
        validation_data=(X_valid, y_valid),
        steps_per_epoch=len(X_train) // batch_size,
        epochs=epochs,
        callbacks=callbacks,
    )

In [ ]:
model.save("model19.h5")

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix


df = pd.read_csv("C:/Users/Pierre/Desktop/final_final_try/fer2013.csv")


emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}


new_model = load_model("model19.h5")


yhat_valid = np.argmax(new_model.predict(X_valid), axis=1)


cm = confusion_matrix(np.argmax(y_valid, axis=1), yhat_valid)


class_names = [emotion_label_to_text[i] for i in range(len(emotion_label_to_text))]


plt.figure(figsize=(7,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig("confusion_matrix_dcnn.png")


total_wrong_predictions = np.sum(np.argmax(y_valid, axis=1) != yhat_valid)
print(f'Total wrong validation predictions: {total_wrong_predictions}\n\n')


print(classification_report(np.argmax(y_valid, axis=1), yhat_valid))
